In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")
num_rows, num_columns = df.shape

Q1

In [3]:
num_columns

19

In [4]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,3.066766e+06,3066766,3066766,2.995023e+06,3.066766e+06,2.995023e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,2.995023e+06,2.995023e+06
mean,1.730215e+00,2023-01-17 00:22:26.288164,2023-01-17 00:38:06.427874,1.362532e+00,3.847342e+00,1.497440e+00,1.663980e+02,1.643926e+02,1.194483e+00,1.836707e+01,1.537842e+00,4.882900e-01,3.367941e+00,5.184907e-01,9.820847e-01,2.702038e+01,2.274231e+00,1.074086e-01
min,1.000000e+00,2008-12-31 23:01:42,2009-01-01 14:29:11,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.000000e+02,-7.500000e+00,-5.000000e-01,-9.622000e+01,-6.500000e+01,-1.000000e+00,-7.510000e+02,-2.500000e+00,-1.250000e+00
25%,1.000000e+00,2023-01-09 16:21:57.250000,2023-01-09 16:37:06,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.540000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2023-01-17 08:42:29.500000,2023-01-17 08:58:30.500000,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.720000e+00,0.000000e+00,1.000000e+00,2.016000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2023-01-24 16:26:27,2023-01-24 16:42:49,1.000000e+00,3.330000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,2023-02-01 00:56:53,2023-02-02 09:28:47,9.000000e+00,2.589281e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.160100e+03,1.250000e+01,5.316000e+01,3.808000e+02,1.969900e+02,1.000000e+00,1.169400e+03,2.500000e+00,1.250000e+00
std,4.438480e-01,NaN,NaN,8.961200e-01,2.495838e+02,6.474767e+00,6.424413e+01,6.994368e+01,5.294762e-01,1.780782e+01,1.789592e+00,1.034641e-01,3.826759e+00,2.017579e+00,1.833529e-01,2.216359e+01,7.718454e-01,3.556511e-01


In [5]:
df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

Q2

In [6]:
from pandas import Timedelta


duration_std: Timedelta = df["duration"].std()
duration_std.total_seconds() / 60

42.594351233333335

In [7]:
df["duration_mins"] = df["duration"].dt.total_seconds() / 60

In [8]:
df = df[(df["duration_mins"] >= 1) & (df["duration_mins"] <= 60)]

Q3

In [9]:
(df.shape[0] / num_rows) * 100

98.1220282212598

Q4

In [11]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()

df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
df_dict = df[["PULocationID", "DOLocationID"]].to_dict(orient="records")
X = dv.fit_transform(df_dict)

In [12]:
X.shape[1]

515

Q5

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

model = LinearRegression()
model.fit(X, df["duration_mins"])

y_pred = model.predict(X)
rmse = root_mean_squared_error(df["duration_mins"], y_pred)

rmse

7.649261952010433

Q6

In [17]:
test_df = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")

test_df["duration"] = test_df["tpep_dropoff_datetime"] - test_df["tpep_pickup_datetime"]
test_df["duration_mins"] = test_df["duration"].dt.total_seconds() / 60

test_df = test_df[(test_df["duration_mins"] >= 1) & (test_df["duration_mins"] <= 60)]

test_df[["PULocationID", "DOLocationID"]] = test_df[["PULocationID", "DOLocationID"]].astype(str)

test_df_dict = test_df[["PULocationID", "DOLocationID"]].to_dict(orient="records")

X_test = dv.transform(test_df_dict)

y_pred_test = model.predict(X_test)

rmse_test = root_mean_squared_error(test_df["duration_mins"], y_pred_test)

rmse_test

7.811817942871299